In [27]:
import re
import unicodedata
import sys


In [28]:

TONE_MARKS = {
    "\u0301": 5,  # sắc
    "\u0300": 2,  # huyền
    "\u0303": 3,  # ngã
    "\u0309": 4,  # hỏi
    "\u0323": 6,  # nặng
}
TONE_DEFAULT = 1  # ngang

def strip_tone_and_get_number(word: str):
    d = unicodedata.normalize("NFD", word)
    tone = None
    out = []
    for ch in d:
        if unicodedata.combining(ch):
            if ch in TONE_MARKS and tone is None:
                tone = TONE_MARKS[ch]
            else:
                out.append(ch)
        else:
            out.append(ch)
    if tone is None:
        tone = TONE_DEFAULT
    return unicodedata.normalize("NFC", "".join(out)), tone

PL_IE  = "\ue000"  # iê/ia/yê -> ie
PL_UO  = "\ue001"  # uô/ua -> uo
PL_UG  = "\ue002"  # ươ/ưa -> ɯɤ
PL_ECI = "\ue003"  # ây -> ɤ̌i

SEQ_RULES = [
    (r"ngh", "ŋ"),
    (r"ng",  "ŋ"),
    (r"gh",  "ɣ"),
    (r"kh",  "χ"),
    (r"ph",  "f"),
    (r"th",  "tʰ"),
    (r"tr",  "ʈ"),
    (r"ch",  "C"),
    (r"gi",  "z"),
    (r"qu",  "kw"),
]

DIPH_RULES = [
    (r"iê", PL_IE), (r"yê", PL_IE), (r"ia", PL_IE),
    (r"uô", PL_UO), (r"ua", PL_UO),
    (r"ươ", PL_UG), (r"ưa", PL_UG),
    (r"ây", PL_ECI),
]

VOW_PRE_RULES = [
    (r"ê", "E"),
    (r"ô", "O"),
    (r"ơ", "ɤ"),
    (r"ư", "ɯ"),
    (r"â", "ɤ̆"),
    (r"ă", "ă"),
]

CONS_SINGLE_RULES = [
    (r"b", "b"),
    (r"c", "k"),
    (r"k", "k"),
    (r"q", "k"),
    (r"đ", "D"),
    (r"g", "ɣ"),
    (r"h", "h"),
    (r"l", "l"),
    (r"m", "m"),
    (r"n", "n"),
    (r"p", "p"),
    (r"r", "ʐ"),
    (r"s", "ʂ"),
    (r"t", "t"),
    (r"v", "v"),
    (r"x", "s"),
    (r"d", "z"),
]

VOW_FINAL_RULES = [
    (r"y", "i"),
    (r"i", "i"),
    (r"e", "ɛ"),
    (r"o", "ɔ"),
    (r"a", "a"),
    (r"u", "u"),
]

def apply_rules(s: str, rules):
    for pat, rep in rules:
        s = re.sub(pat, rep, s)
    return s

def convert_word_with_tone(token: str) -> str:
    if not re.search(r"\w", token, flags=re.UNICODE):
        return token

    base, tone = strip_tone_and_get_number(token)
    s = base.lower()

    s = apply_rules(s, DIPH_RULES)
    s = apply_rules(s, SEQ_RULES)
    s = apply_rules(s, VOW_PRE_RULES)
    s = apply_rules(s, CONS_SINGLE_RULES)
    s = apply_rules(s, VOW_FINAL_RULES)

    s = (s.replace("C", "c")
           .replace("D", "d")
           .replace("E", "e")
           .replace("O", "o"))



    s = (s.replace(PL_IE,  "ie")
           .replace(PL_UO,  "uo")
           .replace(PL_UG,  "ɯɤ")
           .replace(PL_ECI, "ɤ̌i"))

    return f"{s}{tone}"



In [29]:
def vn_to_phonemic(text: str) -> str:
    tokens = re.findall(r"\w+|\W+", text, flags=re.UNICODE)
    return "".join(
        convert_word_with_tone(tok) if re.match(r"^\w+$", tok, flags=re.UNICODE) else tok
        for tok in tokens
    )

In [30]:
vn_to_phonemic("Nếu biết rằng em đã có chồng")

'neu5 biet5 ʐăŋ2 ɛm1 da3 kɔ5 coŋ2'

In [31]:
vn_to_phonemic("trời ơi người ấy có buồn không")

'ʈɤi2 ɤi1 ŋɯɤi2 ɤ̌i5 kɔ5 buon2 χoŋ1'

In [32]:
vn_to_phonemic("Tôi yêu em đến nay chừng có thể. Ngọn lửa tình chưa hẳn đã tàn phai")

'toi1 ieu1 ɛm1 den5 nai1 cɯŋ2 kɔ5 tʰe4. ŋɔn6 lɯɤ4 tinh2 cɯɤ1 hăn4 da3 tan2 fai1'

In [33]:
vn_to_phonemic("Thân em vừa trắng lại vừa tròn")

'tʰɤ̆n1 ɛm1 vɯɤ2 ʈăŋ5 lai6 vɯɤ2 ʈɔn2'